In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# #灰度共生矩阵
# #定义最大灰度级数
# gray_level = 16

# def maxGrayLevel(img):
#     max_gray_level=0
#     (height,width)=img.shape
#     for y in range(height):
#         for x in range(width):
#             if img[y][x] > max_gray_level:
#                 max_gray_level = img[y][x]
#     return max_gray_level+1

# def getGlcm(img,d_x,d_y):
#     srcdata=img.copy()
#     ret=[[0.0 for i in range(gray_level)] for j in range(gray_level)]
#     (height,width) = img.shape

#     max_gray_level=maxGrayLevel(img)

#     #若灰度级数大于gray_level，则将图像的灰度级缩小至gray_level，减小灰度共生矩阵的大小
#     if max_gray_level > gray_level:
#         for j in range(height):
#             for i in range(width):
#                 srcdata[j][i] = srcdata[j][i]*gray_level / max_gray_level

#     for j in range(height-d_y):
#         for i in range(width-d_x):
#             rows = srcdata[j][i]
#             cols = srcdata[j + d_y][i+d_x]
#             ret[rows][cols]+=1.0

#     for i in range(gray_level):
#         for j in range(gray_level):
#             ret[i][j]/=float(height*width)

#     return np.array(ret)
    
# def feature_computer(p):
#     Con=0.0
#     Eng=0.0
#     Asm=0.0
#     Idm=0.0
#     for i in range(gray_level):
#         for j in range(gray_level):
#             Con+=(i-j)*(i-j)*p[i][j]
#             Asm+=p[i][j]*p[i][j]
#             Idm+=p[i][j]/(1+(i-j)*(i-j))
#             if p[i][j]>0.0:
#                 Eng+=p[i][j]*math.log(p[i][j])
#     return Asm,Con,-Eng,Idm

# def batch_GLCM(images):
#     greycomatrix_list_4 = []
#     for i in tqdm(range(len(images))):
#         img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
#         gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
#         glcm_0=getGlcm(gray, 1,0)
#         asm0,con0,eng0,idm0 = feature_computer(glcm_0)
        
#         glcm_1=getGlcm(gray, 0,1)
#         asm1,con1,eng1,idm1 = feature_computer(glcm_1)
        
#         glcm_2=getGlcm(gray, 1,1)
#         asm2,con2,eng2,idm2 = feature_computer(glcm_2)

#         greycomatrix_list_4.append([asm0,con0,eng0,idm0,
#                                   asm1,con1,eng1,idm1,
#                                   asm2,con2,eng2,idm2])
        
#     greycomatrix_list_4 = np.array(greycomatrix_list_4,dtype = float32)
    
#     print (greycomatrix_list_4.shape)
#     return greycomatrix_list_4

In [3]:
# def atoi(text):
#     return int(text) if text.isdigit() else text

# def natural_keys(text):
#     return [ atoi(c) for c in re.split('(\d+)', text) ]

# def load_flower_data():
#     # grab the list of images that we'll be describing
#     print("[INFO] handling images...")
#     TRAIN_ORIGINAL_DIR = '../train/'
#     TRAIN_SUB_DIR = '../subsample/'
#     TRAIN_GAN = '../../image_gan/'
#     TEST_DIR = '../../test/'

#     # use this for full dataset
#     train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
#     test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
#     train_images = train_images_gan
    
#     train_images.sort(key=natural_keys)
#     test_images.sort(key=natural_keys)

#     # initialize the features matrix and labels list
#     trainImage = []
#     trainLabels = []
#     testImage = []
#     testLabels = []

#     # loop over the input images
#     for (i, imagePath) in enumerate(train_images):
#         # extract the class label
#         # get the labels from the name of the images by extract the string before "_"
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # read and resize image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         trainImage.append(img)
#         trainLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
#       # loop over the input images
#     for (i, imagePath) in enumerate(test_images):
#         # extract the class label
#         # our images were named as labels.image_number.format
#         # get the labels from the name of the images by extract the string before "."
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # extract CNN features in the image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         testImage.append(img)
#         testLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


#     trainImage = np.array(trainImage,dtype = float32)
#     trainLabels = np.array(trainLabels)
#     testImage = np.array(testImage,dtype = float32)
#     testLabels = np.array(testLabels)
#     print (trainImage.shape)
    
#     trainImage = trainImage.astype(np.float32) / 255
#     testImage = testImage.astype(np.float32) / 255
    
#     le = preprocessing.LabelEncoder()
#     le.fit(trainLabels)
#     list(le.classes_)
#     trainLabels = le.transform(trainLabels) 
#     testLabels = le.transform(testLabels) 
    
#     return trainImage, trainLabels, testImage, testLabels

In [4]:
# trainImage, trainLabels, testImage, testLabels = load_flower_data()

# trainImage_GLCM_4 = batch_GLCM(trainImage)
# testImage_GLCM_4  = batch_GLCM(testImage)
# nb_classes = 2

# # Convert class vectors to binary class matrices.
# trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
# print (trainLabels)
# testLabels = keras.utils.to_categorical(testLabels, nb_classes)
# print (testLabels)
# print (testLabels.shape)

# np.save('../trainImage.npy', trainImage)
# np.save('../trainLabels.npy', trainLabels)
# np.save('../testImage.npy', testImage)
# np.save('../testLabels.npy', testLabels)
# np.save('../trainImage_GLCM_4.npy', trainImage_GLCM_4)
# np.save('../testImage_GLCM_4.npy', testImage_GLCM_4)

# print("[INFO] trainImage matrix: {:.2f}MB".format(
#     (trainImage.nbytes) / (1024 * 1000.0)))
# print("[INFO] trainLabels matrix: {:.4f}MB".format(
#     (trainLabels.nbytes) / (1024 * 1000.0)))
# print("[INFO] testImage matrix: {:.2f}MB".format(
#     (testImage.nbytes) / (1024 * 1000.0)))
# print("[INFO] testLabels matrix: {:.4f}MB".format(
#     (testLabels.nbytes) / (1024 * 1000.0)))
# print("[INFO] trainImage_GLCM_4 matrix: {:.2f}MB".format(
#     (trainImage_GLCM_4.nbytes) / (1024 * 1000.0)))
# print("[INFO] testImage_GLCM_4 matrix: {:.4f}MB".format(
#     (testImage_GLCM_4.nbytes) / (1024 * 1000.0)))

In [5]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name\
        
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.glcm_dim = 12  # glcm dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_GLCM_4_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        #定义最大灰度级数
        self.gray_level = 128
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_glcm = np.load('../trainImage_GLCM_4.npy')
        self.test_x_glcm = np.load('../testImage_GLCM_4.npy')
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_GLCM, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_GLCM",x_GLCM.get_shape()) # 128, 128, 3 
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_GLCM,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_GLCM_fill = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.glcm_dim], 
                                name='x_GLCM_fill')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_GLCM_fill, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.x_GLCM_fill, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_glcm = self.train_x_glcm[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
#                 batch_x_GLCM = self.batch_GLCM(batch_x)
                batch_x_GLCM = shuffled_set_glcm[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, 1, 1, self.glcm_dim])
                batch_x_GLCM_fill = batch_x_GLCM * np.ones([self.batch_size, self.input_height, self.input_width,
                                                            self.glcm_dim])
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_GLCM_fill: batch_x_GLCM_fill,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
#                 batch_x_GLCM_test =self.batch_GLCM(batch_x_test)
                batch_x_GLCM_test = self.test_x_glcm[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, 1, 1, self.glcm_dim])
                batch_x_GLCM_fill_test = batch_x_GLCM_test * np.ones([self.batch_size, self.input_height, self.input_width, self.glcm_dim])
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_GLCM_fill: batch_x_GLCM_fill_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_GLCM_test = self.test_x_glcm[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, 1, 1, self.glcm_dim])
            batch_x_GLCM_fill_test = batch_x_GLCM_test * np.ones([self.batch_size, self.input_height, 
                                                                  self.input_width, self.glcm_dim])
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_GLCM_fill: batch_x_GLCM_fill_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_GLCM_test = self.test_x_glcm[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, 1, 1, self.glcm_dim])
            batch_x_GLCM_fill_test = batch_x_GLCM_test * np.ones([self.batch_size, self.input_height, 
                                                                  self.input_width, self.glcm_dim])
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_GLCM_fill: batch_x_GLCM_fill_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def maxGrayLevel(self, img):
        max_gray_level=0
        (height,width)=img.shape
        for y in range(height):
            for x in range(width):
                if img[y][x] > max_gray_level:
                    max_gray_level = img[y][x]
        return max_gray_level+1

    def getGlcm(self, img,d_x,d_y):
        srcdata=img.copy()
        ret=[[0.0 for i in range(self.gray_level)] for j in range(self.gray_level)]
        (height,width) = img.shape

        max_gray_level=self.maxGrayLevel(img)

        #若灰度级数大于gray_level，则将图像的灰度级缩小至gray_level，减小灰度共生矩阵的大小
        if max_gray_level > self.gray_level:
            for j in range(height):
                for i in range(width):
                    srcdata[j][i] = srcdata[j][i]*self.gray_level / max_gray_level

        for j in range(height-d_y):
            for i in range(width-d_x):
                rows = srcdata[j][i]
                cols = srcdata[j + d_y][i+d_x]
                ret[rows][cols]+=1.0

        for i in range(self.gray_level):
            for j in range(self.gray_level):
                ret[i][j]/=float(height*width)

        return np.array(ret)
    
    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [ ]:
dataset = '4_Flowers_1s'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
#  [*] Best Epoch:  33 , Accuracy:  0.7799999713897705
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_GLCM_4_C5_D1_Kernel(3,3)_128_lrdecay/CNN_GLCM_4_C5_D1_Kernel(3,3)_128_lrdecay-33
#  [*] Finished testing Best Epoch: 33 , accuracy:  0.7799999713897705 !

# Avg per epoch ptime: 8.29, total 50 epochs ptime: 432.88
#  [*] Training finished!
#  [*] Best Epoch:  1 , Accuracy:  0.7999999523162842
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_GLCM_4_C5_D1_Kernel(3,3)_128_lrdecay/CNN_GLCM_4_C5_D1_Kernel(3,3)_128_lrdecay-1
#  [*] Finished testing Best Epoch: 1 , accuracy:  0.7999999523162842 !

CNN:x (100, 128, 128, 3)
CNN:x_GLCM (100, 128, 128, 12)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_GLCM (100, 128, 128, 12)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_1_2/kernel:0 (float32_ref 3x3x12x32) [3456, bytes: 13824]
cnn/con

Epoch: [ 5] [  16/  20] time: 42.8721, loss: 0.08398198
Epoch: [ 5] [  17/  20] time: 43.2751, loss: 0.02445393
Epoch: [ 5] [  18/  20] time: 43.6779, loss: 0.04666627
Epoch: [ 5] [  19/  20] time: 44.0768, loss: 0.02884663
[5/50] - ptime: 8.3224 loss: 0.02606603 acc: 0.79000 lr: 0.00100000
Epoch: [ 6] [   0/  20] time: 45.2517, loss: 0.01076504
Epoch: [ 6] [   1/  20] time: 45.6535, loss: 0.00703770
Epoch: [ 6] [   2/  20] time: 46.0488, loss: 0.00626811
Epoch: [ 6] [   3/  20] time: 46.4450, loss: 0.02275191
Epoch: [ 6] [   4/  20] time: 46.8435, loss: 0.03972794
Epoch: [ 6] [   5/  20] time: 47.2474, loss: 0.03683512
Epoch: [ 6] [   6/  20] time: 47.6525, loss: 0.00563183
Epoch: [ 6] [   7/  20] time: 48.0551, loss: 0.09456391
Epoch: [ 6] [   8/  20] time: 48.4510, loss: 0.03872626
Epoch: [ 6] [   9/  20] time: 48.8377, loss: 0.04416813
Epoch: [ 6] [  10/  20] time: 49.2223, loss: 0.02481940
Epoch: [ 6] [  11/  20] time: 49.6103, loss: 0.01568204
Epoch: [ 6] [  12/  20] time: 49.996

Epoch: [12] [  14/  20] time: 102.6267, loss: 0.00164070
Epoch: [12] [  15/  20] time: 103.0240, loss: 0.01599712
Epoch: [12] [  16/  20] time: 103.4199, loss: 0.00446005
Epoch: [12] [  17/  20] time: 103.8177, loss: 0.00863937
Epoch: [12] [  18/  20] time: 104.2131, loss: 0.00065396
Epoch: [12] [  19/  20] time: 104.6098, loss: 0.02442282
[12/50] - ptime: 8.2557 loss: 0.00834691 acc: 0.73000 lr: 0.00090000
Epoch: [13] [   0/  20] time: 105.6959, loss: 0.00338952
Epoch: [13] [   1/  20] time: 106.0936, loss: 0.01117324
Epoch: [13] [   2/  20] time: 106.4951, loss: 0.02626865
Epoch: [13] [   3/  20] time: 106.8962, loss: 0.00153270
Epoch: [13] [   4/  20] time: 107.2960, loss: 0.00021074
Epoch: [13] [   5/  20] time: 107.7063, loss: 0.00011741
Epoch: [13] [   6/  20] time: 108.1118, loss: 0.00623184
Epoch: [13] [   7/  20] time: 108.5078, loss: 0.01260487
Epoch: [13] [   8/  20] time: 108.9051, loss: 0.00134460
Epoch: [13] [   9/  20] time: 109.3008, loss: 0.01408030
Epoch: [13] [  10/ 

Epoch: [19] [  10/  20] time: 161.6391, loss: 0.00059184
Epoch: [19] [  11/  20] time: 162.0278, loss: 0.02454558
Epoch: [19] [  12/  20] time: 162.4352, loss: 0.00185023
Epoch: [19] [  13/  20] time: 162.8309, loss: 0.00404632
Epoch: [19] [  14/  20] time: 163.2271, loss: 0.00722813
Epoch: [19] [  15/  20] time: 163.6271, loss: 0.00141096
Epoch: [19] [  16/  20] time: 164.0227, loss: 0.02632907
Epoch: [19] [  17/  20] time: 164.4248, loss: 0.00130809
Epoch: [19] [  18/  20] time: 164.8284, loss: 0.03346567
Epoch: [19] [  19/  20] time: 165.2249, loss: 0.00711981
[19/50] - ptime: 8.2815 loss: 0.01067362 acc: 0.59000 lr: 0.00090000
Epoch: [20] [   0/  20] time: 166.2944, loss: 0.01796000
Epoch: [20] [   1/  20] time: 166.6967, loss: 0.00154205
Epoch: [20] [   2/  20] time: 167.0997, loss: 0.00595906
Epoch: [20] [   3/  20] time: 167.4953, loss: 0.00665877
Epoch: [20] [   4/  20] time: 167.8897, loss: 0.00090729
Epoch: [20] [   5/  20] time: 168.2859, loss: 0.00000515
Epoch: [20] [   6/ 

Epoch: [26] [   6/  20] time: 220.4361, loss: 0.00069824
Epoch: [26] [   7/  20] time: 220.8320, loss: 0.00004370
Epoch: [26] [   8/  20] time: 221.2295, loss: 0.00054956
Epoch: [26] [   9/  20] time: 221.6270, loss: 0.00421159
Epoch: [26] [  10/  20] time: 222.0101, loss: 0.00106613
Epoch: [26] [  11/  20] time: 222.3992, loss: 0.00167091
Epoch: [26] [  12/  20] time: 222.7835, loss: 0.00002996
Epoch: [26] [  13/  20] time: 223.1792, loss: 0.00526634
Epoch: [26] [  14/  20] time: 223.5788, loss: 0.00050604
Epoch: [26] [  15/  20] time: 223.9746, loss: 0.00068766
Epoch: [26] [  16/  20] time: 224.3695, loss: 0.00044894
Epoch: [26] [  17/  20] time: 224.7662, loss: 0.02820353
Epoch: [26] [  18/  20] time: 225.1615, loss: 0.00004946
Epoch: [26] [  19/  20] time: 225.5497, loss: 0.02161461
[26/50] - ptime: 8.2119 loss: 0.00344355 acc: 0.57000 lr: 0.00081000
Epoch: [27] [   0/  20] time: 226.6091, loss: 0.00161061
Epoch: [27] [   1/  20] time: 227.0056, loss: 0.00001493
Epoch: [27] [   2/ 

Epoch: [33] [   2/  20] time: 279.2461, loss: 0.00178625
Epoch: [33] [   3/  20] time: 279.6445, loss: 0.00038757
Epoch: [33] [   4/  20] time: 280.0405, loss: 0.00498284
Epoch: [33] [   5/  20] time: 280.4370, loss: 0.00106574
Epoch: [33] [   6/  20] time: 280.8340, loss: 0.00106844
Epoch: [33] [   7/  20] time: 281.2300, loss: 0.00009658
Epoch: [33] [   8/  20] time: 281.6339, loss: 0.00012258
Epoch: [33] [   9/  20] time: 282.0286, loss: 0.00016265
Epoch: [33] [  10/  20] time: 282.4227, loss: 0.00037253
Epoch: [33] [  11/  20] time: 282.8188, loss: 0.00065994
Epoch: [33] [  12/  20] time: 283.2137, loss: 0.00076691
Epoch: [33] [  13/  20] time: 283.6087, loss: 0.00001889
Epoch: [33] [  14/  20] time: 284.0037, loss: 0.00170172
Epoch: [33] [  15/  20] time: 284.3985, loss: 0.00054808
Epoch: [33] [  16/  20] time: 284.7925, loss: 0.00006413
Epoch: [33] [  17/  20] time: 285.1868, loss: 0.00011419
Epoch: [33] [  18/  20] time: 285.5855, loss: 0.00542350
Epoch: [33] [  19/  20] time: 2

Epoch: [39] [  19/  20] time: 337.5905, loss: 0.00011917
[39/50] - ptime: 8.3098 loss: 0.00015776 acc: 0.56000 lr: 0.00072900
Epoch: [40] [   0/  20] time: 338.6793, loss: 0.00001455
Epoch: [40] [   1/  20] time: 339.0758, loss: 0.00005765
Epoch: [40] [   2/  20] time: 339.4705, loss: 0.00004717
Epoch: [40] [   3/  20] time: 339.8641, loss: 0.00008811
Epoch: [40] [   4/  20] time: 340.2603, loss: 0.00000447
Epoch: [40] [   5/  20] time: 340.6570, loss: 0.00000724
Epoch: [40] [   6/  20] time: 341.0567, loss: 0.00011518
Epoch: [40] [   7/  20] time: 341.4522, loss: 0.00001452
Epoch: [40] [   8/  20] time: 341.8527, loss: 0.00017789
Epoch: [40] [   9/  20] time: 342.2485, loss: 0.00000874
Epoch: [40] [  10/  20] time: 342.6438, loss: 0.00009174
Epoch: [40] [  11/  20] time: 343.0388, loss: 0.00000884
Epoch: [40] [  12/  20] time: 343.4332, loss: 0.00000441
Epoch: [40] [  13/  20] time: 343.8269, loss: 0.00001130
Epoch: [40] [  14/  20] time: 344.2276, loss: 0.00021691
Epoch: [40] [  15/ 

Epoch: [46] [  15/  20] time: 396.2101, loss: 0.00001864
Epoch: [46] [  16/  20] time: 396.6049, loss: 0.00001300
Epoch: [46] [  17/  20] time: 396.9914, loss: 0.00000419
Epoch: [46] [  18/  20] time: 397.3736, loss: 0.00000306
Epoch: [46] [  19/  20] time: 397.7710, loss: 0.00000646
[46/50] - ptime: 8.2079 loss: 0.00005135 acc: 0.51000 lr: 0.00065610
Epoch: [47] [   0/  20] time: 398.8228, loss: 0.00000001
Epoch: [47] [   1/  20] time: 399.2238, loss: 0.00000087
Epoch: [47] [   2/  20] time: 399.6265, loss: 0.00036327
Epoch: [47] [   3/  20] time: 400.0236, loss: 0.00004539
Epoch: [47] [   4/  20] time: 400.4170, loss: 0.00000440
Epoch: [47] [   5/  20] time: 400.8112, loss: 0.00011001
Epoch: [47] [   6/  20] time: 401.2113, loss: 0.00031850
Epoch: [47] [   7/  20] time: 401.6141, loss: 0.00000004
Epoch: [47] [   8/  20] time: 402.0116, loss: 0.00001933
Epoch: [47] [   9/  20] time: 402.4091, loss: 0.00001315
Epoch: [47] [  10/  20] time: 402.8223, loss: 0.00003156
Epoch: [47] [  11/ 

In [ ]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放